### 1.Analyze the distribution of the data provided in the dataset and draw any conclusions:


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("dataset_SCL.csv")

# Analyze the distribution of the data
print(df.describe())
# Draw any conclusions based on the distribution of the data


In [ ]:
df.head()


In [ ]:
df.tail()


In [ ]:
df.info()


In [ ]:
df.shape


In [ ]:
df.columns


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("dataset_SCL.csv")

# Statistical summary of the data
print(df.describe())

# Information about the data
print(df.info())

# Frequency of each unique value in a particular column
print(df['TIPOVUELO'].value_counts())

# Mean of a particular column based on a grouping column
print(df.groupby('OPERA')['AÑO'].mean())

# Use visualization libraries
# Histogram
df.hist(bins=50, figsize=(20, 15))
plt.show()

# Bar plot
sns.countplot(x='DIA', data=df)
plt.show()

# Correlation matrix and heatmap
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True)
plt.show()

# Cross-tabulation
pd.crosstab(df['SIGLADES'], df['TIPOVUELO'])


### 2.Create additional columns and export them in a file called "synthetic_features.csv":


In [ ]:
import numpy as np

df['Fecha-I'] = pd.to_datetime(df['Fecha-I'])
df['Fecha-O'] = pd.to_datetime(df['Fecha-O'])

# Create the 'temporada_alta' column
df['temporada_alta'] = np.where(((df['Fecha-I'].dt.month == 12) & (df['Fecha-I'].dt.day >= 15)) |
                                ((df['Fecha-I'].dt.month == 3) & (df['Fecha-I'].dt.day <= 3)) |
                                ((df['Fecha-I'].dt.month == 7) & (df['Fecha-I'].dt.day >= 15)) |
                                ((df['Fecha-I'].dt.month == 7) & (df['Fecha-I'].dt.day <= 31)) |
                                ((df['Fecha-I'].dt.month == 9) & (df['Fecha-I'].dt.day >= 11)) |
                                ((df['Fecha-I'].dt.month == 9) & (df['Fecha-I'].dt.day <= 30)), 1, 0)

# Create the 'dif_min' column
df['dif_min'] = (df['Fecha-O'] - df['Fecha-I']).dt.total_seconds() / 60

# Create the 'atraso_15' column
df['atraso_15'] = np.where(df['dif_min'] > 15, 1, 0)

# Create the 'periodo_dia' column
df['periodo_dia'] = np.where((df['Fecha-I'].dt.hour >= 5) & (df['Fecha-I'].dt.hour < 12), 'morning',
                             np.where((df['Fecha-I'].dt.hour >= 12) & (df['Fecha-I'].dt.hour < 19), 'afternoon', 'night'))

# Export the new columns to a file
df[['temporada_alta', 'dif_min', 'atraso_15', 'periodo_dia']].to_csv(
    "synthetic_features.csv", index=False)


3.Analyze the delay rate by destination, airline, month of the year, day of the week, season, and type of flight:

In [ ]:
# df["Fecha-I"] = pd.to_datetime(df["Fecha-I"])

# df["DIANOM"] = df["Fecha-I"].dt.weekday_name
# df["DIA"] = df["Fecha-I"].dt.day
# df["MES"] = df["Fecha-I"].dt.month
# df["AÑO"] = df["Fecha-I"].dt.year


# Analyze the delay rate by destination
print(df.groupby('SIGLADES')['atraso_15'].mean())

# Analyze the delay rate by airline. Print with a blank line after all the results.
print(df.groupby('OPERA')['atraso_15'].mean())

# Rate by month of the year
print(df.groupby(df['Fecha-I'].dt.month)['atraso_15'].mean())

# Analyze the delay rate by day of the week
print(df.groupby(df['Fecha-I'].dt.day)['atraso_15'].mean())

# Analyze the delay rate by season
print(df.groupby('temporada_alta')['atraso_15'].mean())

# Analyze the delay rate by type of flight
print(df.groupby('TIPOVUELO')['atraso_15'].mean())

1. 'DIANOM': The day of the week can impact the likelihood of delays, as certain days

may have higher traffic or more unpredictable weather.

2. 'MES': Some months may have more extreme weather conditions or higher travel

demand, which could impact the likelihood of delays.

3. 'AÑO': The year may also be important, as economic and political factors can affect the

number of flights and travel demand.

4. 'OPERA': Different airlines may have different operational efficiencies and safety

records, which can impact the likelihood of delays.

5. 'SIGLAORI*: The city of origin may also be important, as certain cities may have more

unpredictable weather or higher demand for flights.

6. 'SIGLADES': The city of destination may also be important, as certain destinations may

have more unpredictable weather or higher demand for flights.

7. 'TIPOVUELO': the type of flight, international or national, can impact the likelihood of

delays.

8. 'DIA': Day of the month, certain days of the month may have higher traffic.

9. 'temporada _alta': High season, travel season may have more flights, more demand,

and more chances of delays.

10. 'dif_min': Time difference between scheduled and operated flights, more difference

may have more chances of delays.

11. 'periodo _dia': time of the flight, morning, evening or night flights may have different

chances of delays.

### To determine which variables are most important in predicting delays, we can use various machine learning techniques like linear or logistic regression, Random Forest, Decision Trees, etc.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

'''ValueError: could not convert string to float: 'Martes'
This error is caused by the fact that the LinearRegression model in scikit-learn only accepts numerical data, but some of the columns in your dataframe contain non-numerical values such as 'Martes' or 'Internacional'.
To fix this error, you will need to convert the non-numerical values in these columns to numerical values. One way to do this is by using the pandas function 'get_dummies' which converts categorical variables into numerical variables.
Here is an example of how you can use get_dummies to convert 'DIANOM', 'OPERA', 'SIGLAORI', 'SIGLADES' and 'TIPOVUELO' columns to numerical values:'''

# Prepare the data
X = df[['MES', 'AÑO', 'DIA', 'temporada_alta', 'dif_min', 'periodo_dia']]
X = pd.get_dummies(df[['DIANOM', 'OPERA', 'SIGLAORI', 'SIGLADES', 'TIPOVUELO']], drop_first=True)
y = df['atraso_15']

# X = df[['DIANOM', 'MES', 'AÑO', 'OPERA', 'SIGLAORI', 'SIGLADES', 'TIPOVUELO', 'DIA', 'temporada_alta', 'dif_min', 'periodo_dia']]
# y = df['atraso_15']

'''In this code snippet, we first select the numerical columns of our dataframe and then use get_dummies function to convert the non-numerical columns to numerical columns. We pass the dataframe containing the non-numerical columns to the get_dummies function, and set the drop_first parameter to True to remove one of the dummy variables for each categorical feature and reduce multicollinearity.
Also, you can use the LabelEncoder from sklearn.preprocessing to convert non-numerical values to numerical values, it works by assigning a unique integer to each unique value in a column.'''

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['DIANOM'] = le.fit_transform(df['DIANOM'])
# df['OPERA'] = le.fit_transform(df['OPERA'])
# df['SIGLAORI'] = le.fit_transform(df['SIGLAORI'])
# df['SIGLADES'] = le.fit_transform(df['SIGLADES'])
# df['TIPOVUELO'] = le.fit_transform(df['TIPOVUELO'])


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the linear regression model
reg = LinearRegression()
reg.fit(X_train, y_train)

# Get the feature importance coefficients
coefs = reg.coef_

# Create a dictionary to store the feature importance scores
feature_importance = dict(zip(X.columns, coefs))

# Print the feature importance scores
print(feature_importance)

# Predict the test set labels
y_pred = reg.predict(X_test)

# Evaluate the model
print("Accuracy" + str(reg.score(X_test, y_test)))



### The above code uses linear regression to predict the 'atraso_15' column using the other columns in the dataset, and it calculates the importance of each feature by the coefficient of each feature in the linear regression model.

### Evaluate the model and draw conclusions:

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred = reg.predict(X_test)

# Get the MSE of the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


##### When assessing model performance, it is important to choose the right metrics depending on the problem type and the characteristics of the data.

In the case of a classification problem, some common metrics are accuracy, precision, recall, F1-score, and AUC-ROC.

Accuracy: measures the proportion of correct predictions among all predictions.
Precision: measures the proportion of true positive predictions among all positive predictions.
Recall: measures the proportion of true positive predictions among all actual positive instances.
F1-score: is the harmonic mean of precision and recall, and it gives a balance between the two.
AUC-ROC: measures the ability of the model to distinguish between positive and negative classes, it is a good metric to use when there is an imbalance in the classes.
In the case of a regression problem, some common metrics are mean squared error, mean absolute error, R-squared, and explained variance.

Mean squared error: measures the average of the squared differences between the predicted and actual values.
Mean absolute error: measures the average of the absolute differences between the predicted and actual values.
R-squared: measures the proportion of the variation in the target variable that is explained by the features.
Explained variance: measures the proportion of the variation in the target variable that is explained by the features.
Once you have evaluated the performance of the model using the appropriate metrics, you can pick the best trained model and evaluate the following:

Influential variables: You can use feature importance scores obtained from the model, or the coefficients of the features obtained by using linear regression, or using the feature importance score obtained by using Random Forest, Decision Trees etc, to determine which variables were most influential in the prediction task.
Performance improvement: You can try different techniques such as feature engineering, hyperparameter tuning, ensemble models, data preprocessing, and using more data to improve the performance of the model.
It is important to keep in mind that in some cases, the best model can be an ensemble of multiple models, or the performance of a model may be improved by a combination of different techniques.